### 5.3.1 Word2Vec 利用事例

#### 前提

次のライブラリがpipコマンドで導入済みであること

```
pip install wikipedia
pip install janome
pip install gensim
conda install keras
```

In [ ]:
# リスト 5.3.1 初期処理と変数宣言

# Macの問題回避
import os
import platform
if platform.system() == 'Darwin':
    os.environ['KMP_DUPLICATE_LIB_OK']='True'
    
# waring抑止
import warnings
warnings.filterwarnings('ignore')

# Word2Vecの隠れ層ノード数
EMBEDDING_DIM = 300

# LSTMで保持する時系列データ数
MAX_LEN = 50

In [ ]:
# リスト5.3.2 テキスト取得
# wikipedia ライブラリを利用

import wikipedia
wikipedia.set_lang("ja")

# 学習データ(歴史)のタイトル
list1 = ['大和時代', '奈良時代', '平安時代', '鎌倉時代', '室町時代', '安土桃山時代', '江戸時代',
        '藤原道長','平清盛','源頼朝', '北条早雲','伊達政宗','徳川家康', '武田信玄', '上杉謙信', 
         '今川義元', '毛利元就', '足利尊氏', '足利義満', '北条泰時']

# 学習データ(地理)のタイトル
list2 = ['東北地方', '関東地方', '中部地方', '近畿地方', '中国地方', '四国地方', '九州地方',
        '北海道', '秋田県', '福島県', '宮城県', '新潟県', '長野県', '山梨県', '静岡県', '愛知県', 
         '栃木県', '群馬県', '千葉県', '神奈川県']

# テストデータのタイトル
list3 = ['織田信長', '豊臣秀吉', '青森県', '北海道']

# それぞれのタイトルに対してWikipediaのsummaryを取得し、配列に保存
list1_w = [wikipedia.summary(item) for item in list1]
list2_w = [wikipedia.summary(item) for item in list2]
list3_w = [wikipedia.summary(item) for item in list3]

# すべての取得結果を 1つのlistに集約
list_all_w = list1_w + list2_w + list3_w

In [ ]:
# リスト 5.3.3 テキストに対して単語毎にブランクを入れる
# 形態素解析としてJanomeを利用

from janome.tokenizer import Tokenizer
t = Tokenizer()
def wakati(text):
    w = t.tokenize(text, wakati=True)
    return ' '.join(w)

list1_x = [wakati(w) for w in list1_w]
list2_x = [wakati(w) for w in list2_w]
list3_x = [wakati(w) for w in list3_w]
list_all_x = list1_x + list2_x + list3_x

In [ ]:
# リスト 5.3.4 学習データ作成

import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()

# 学習・検証で使う全テキストを引数にして辞書を作成する
tokenizer.fit_on_texts(list_all_x)

# 単語一覧を取得
word_index = tokenizer.word_index

# 総単語数を取得
num_words = len(word_index)
print('総単語数: ', num_words)

# 変換前の検証用テキスト確認
print('変換前テキスト: ', list3_x[0])

# テキストの数値化
sequence_test = tokenizer.texts_to_sequences(list3_x)

# 変換結果確認
print('変換後: ', sequence_test[0])

# 単語のパディング(短いときは0で埋める、長いときは途中で切る)
sequence_test = pad_sequences(sequence_test, maxlen=MAX_LEN)

# 変換結果確認
print('パディング後: ', sequence_test[0])

# 学習データに対しても同じ変換を行う
sequence_train = tokenizer.texts_to_sequences(list1_x + list2_x)
sequence_train = pad_sequences(sequence_train, maxlen=MAX_LEN)

# 正解データ作成
# y = 0: 歴史  
# y = 1: 地理  
    
Y_train = np.array([0] * len( list1_x) + [1] * len( list2_x))
Y_test = np.array([0] * 2 + [1] * 2)
print('正解データ(学習用): ', Y_train)
print('正解データ(検証用): ', Y_test)

In [ ]:
# リスト 5.3.5 学習済みWord2Vecロード

# ja.binファイルの準備方法は5.2節を参照のこと

import gensim 
word_vectors = gensim.models.Word2Vec.load('ja.bin')
print(word_vectors)

In [ ]:
# リスト 5.3.6 Embedding Matrix作成
import numpy as np

# num_words: 辞書作成時に検出された単語数 (リスト 5.3.4)
# EMBEDDING_DIM: Word2Vec 隠れ層のノード数 (リスト 5.3.1)
# メモリー領域節約のため、float32を利用します
embedding_matrix = np.zeros((num_words+1, EMBEDDING_DIM), dtype=np.float32)

# Embedding MatrixにWord2Vecの重みベクトル値をコピー
for word, i in word_index.items():
    if word in word_vectors.wv.vocab:
        embedding_matrix[i] = word_vectors[word]

In [ ]:
# リスト 5.3.7 LSTMモデル作成

from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

model = Sequential()
model.add(Embedding(num_words+1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    trainable=False))
model.add(LSTM(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# リスト 5.3.8 学習

# モデル学習の実施
model.fit(sequence_train, Y_train,validation_data=(
    sequence_test, Y_test), batch_size=128, verbose=1, epochs=100)

In [ ]:
# リスト 5.3.9 予測・評価

# 検証データの内容
for text in list3_w:
    print(text)

# 評価
model.predict(sequence_test)